Сделаю упражнение 8.3

In [87]:
import numpy as np
import pandas as pd

In [88]:
cough = np.array([0, 1, 1, 1, 1, 0, 0, 0])
temperature = np.array([1, 1, 0, 1, 0, 1, 1, 0])
breath = np.array([1, 0, 1, 1, 0, 1, 0, 0])
tedious = np.array([1, 1, 1, 0, 1, 0, 0, 1])
sick = np.array([1, 1, 1, 1, 0, 0, 0, 0])
print(cough.shape, temperature.shape, breath.shape, tedious.shape, sick.shape)

(8,) (8,) (8,) (8,) (8,)


In [89]:
data = pd.DataFrame(dict(cough = np.array([0, 1, 1, 1, 1, 0, 0, 0]),
                         temperature = np.array([1, 1, 0, 1, 0, 1, 1, 0]),
                         breath = np.array([1, 0, 1, 1, 0, 1, 0, 0]),
                         tedious = np.array([1, 1, 1, 0, 1, 0, 0, 1]),
                         sick = np.array([1, 1, 1, 1, 0, 0, 0, 0])
                         ), index=np.arange(1, len(cough) + 1))
data.index.name = "Patient"
data

,cough,temperature,breath,tedious,sick
Patient,,,,,
1,0,1,1,1,1
2,1,1,0,1,1
3,1,0,1,1,1
4,1,1,1,0,1
5,1,0,0,1,0
6,0,1,1,0,0
7,0,1,0,0,0
8,0,0,0,1,0


Создадим наивный Байесовский классификатор

In [90]:
def P_A(data):# априорная вероятность заболеть
    return np.sum(data["sick"]) / len(data)

def P_BA(feature_name, data, yes):# вероятность встретить признак у больного
    num_of_sick = len(data[data["sick"] == 1])
    num_of_sick_with_feature = len(data[(data[feature_name] == yes) & (data["sick"] == 1)])
    return num_of_sick_with_feature/ num_of_sick

def P_B_antiA(feature_name, data, yes): # вероятность встретить признак у здорового
    num_of_healthy = len(data[data["sick"] == 0])
    num_of_healthy_with_feature = len(data[(data[feature_name] == yes) & (data["sick"] == 0)])
    return num_of_healthy_with_feature/ num_of_healthy


In [91]:
def naive_bayes(feature, data, yes=1):
    num_of_healthy = len(data) - len(data[data["sick"] == 0])

    aprior = P_A(data)
    pba = P_BA(feature, data, yes)
    p_no_a = P_B_antiA(feature, data, yes) * num_of_healthy / len(data)
    return aprior * pba / (pba * aprior + p_no_a) 

def naive_bayes_for_many(features, data, yes_array):
    aprior = P_A(data)
    ans = aprior # изначально ответ равен априорной вероятности
    p_with_a = [1]
    p_without_a = [1]
    for i in range(len(features)):
        feature = features[i]
        yes = yes_array[i]
        p_with_a.append(P_BA(feature, data, yes))
        p_without_a.append(P_B_antiA(feature, data, yes))
    return ans * np.prod(p_with_a) / (np.prod(p_with_a) * aprior + np.prod(p_without_a) * (1 - aprior))


Пункты А и Б делаем с помощью классификатора для одного признака

In [92]:
print(naive_bayes("cough", data))
print(naive_bayes("tedious",data, yes=0))

0.75
0.3333333333333333


Пункты В и Г делаем функцией для нескольких признаков

In [93]:
features = ["cough", "temperature"]
yes_array = [1, 1]
print(naive_bayes_for_many(features, data, yes_array))

0.8181818181818182


In [94]:
features = ["cough", "temperature", "breath"]
yes_array = [1, 1, 0]
print(naive_bayes_for_many(features, data, yes_array))

0.6


Получили что ответы сошлись с книгой, но код по хорошему стоит переделать